In [ ]:
#_________________________________________________________________________________________________________________________#
#______________________________________________________Raven.ai___________________________________________________________#
#_________________________________________________________________________________________________________________________#

import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt


In [ ]:
def func(x, y):
    return - 5.5 * tf.exp(- 20.0 * (x - 0.3)**2 - 40.0 * (y - 0.3)**2) - 5 * tf.exp(- 15.0 * (x - 0.6)**2 - 10.0 * (y - 0.85)**2) - 2.0 * tf.sin(2.0 * (x - y))  

In [ ]:
# We need some data !! let's create 400 point between 0 and 1 and then transform the outpur into square matrix with X, Y values, finaly apply func for some curvature variation

x = np.linspace(0, 1, 400)
X, Y = np.meshgrid(x, x)
Z = func(X, Y)

In [ ]:
# Okay l'est plot our data to see how we can tackle it with SGD

fig = go.Figure(
    data=[
          go.Surface(
              z=Z, 
              x=X, 
              y=Y, 
              colorscale='viridis'
              )
          ]
        )
fig.update_layout(autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# we got a nice data surface with local and global minima

In [ ]:
# l'ets define our variables and optimizers and capture the values for each step
def constr(a, b):
    assert b > a
    return lambda x: tf.clip_by_value(x, a, b)


x = tf.Variable(0.0, trainable=True, dtype=tf.float64, name='x', constraint=constr(0, 1))
y = tf.Variable(0.0, trainable=True, dtype=tf.float64, name='y', constraint=constr(0, 1))


def objective():
    return - 5.5 * tf.exp(- 20.0 * (x - 0.3)**2 - 40.0 * (y - 0.3)**2) - 3.5 * tf.exp(- 15.0 * (x - 0.6)**2 - 10.0 * (y - 0.85)**2) - 2.0 * tf.sin(2.0 * (x - y))


def optimize(start, verbose=False, method='SGD'):
    x.assign(start[0])
    y.assign(start[1])

    optimizers = tf.keras.optimizers.SGD(learning_rate=0.003)
    obj_vals = []
    coords = []

    for i in range(50):
        if verbose and i % 5 == 0:
            print(f'step: {i}, obj = {objective().numpy():.4f}, x = {x.numpy():.4f}, y = {y.numpy():.4f}')
        obj_vals.append(objective().numpy())
        coords.append((x.numpy(), y.numpy()))
        optimizers.minimize(objective, var_list=[x, y])
        
    return obj_vals, coords

In [ ]:
def plot(obj_vals, coords):

  xcoord = [x[0] for x in coords]
  ycoord = [x[1] for x in coords]
  
  fig = go.Figure()

  fig.add_trace(
      go.Surface(  
          z=Z,
          y=Y,
          x=X,
          colorscale='viridis',
      )
  )

  fig.add_trace(
      go.Scatter3d(
          x = xcoord,
          y = ycoord,
          z = obj_vals,
          mode ='markers',
          marker = dict(
              size = 5,
              color = "#A3336C",
          )
      )
  )

  fig.show()

In [ ]:
obj_vals, coords = optimize([0.25, 0.65], verbose=True, method='SGD')
plot(obj_vals, coords)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# hummm we can see that we are stuck on local minima and the SGD already converged on wrong solution, so to handdle this we have many options
# chosing another starting point
# tunning the learning rate 
# changing optimiser

In [ ]:
# here we just set another starting point to help our SGD to avoid the local minima

obj_vals, coords = optimize([0.2, 0.65], method='SGD')  
plot(obj_vals, coords)

Output hidden; open in https://colab.research.google.com to view.